In [2]:
import xarray as xr
import glob
import pandas as pd
import numpy as np

### Meteo

#### Read all files

In [8]:
geo_folder = '../geo_data/great_db'

In [10]:
# little helper function
def xr_opener(file):
    ds = xr.open_dataset(file)
    if 'index' in ds.coords:
        ds = ds.rename({'index': 'date'}) 
    return ds

files = [file for file in glob.glob(f'{geo_folder}/nc_concat/*.nc')
         if file.split('/')[-1][:-3] in ['2166', '2177']]
big_file = xr.concat([xr_opener(file) for file in files],
                     dim='gauge_id')
big_file


<xarray.Dataset>
Dimensions:     (date: 4749, gauge_id: 2)
Coordinates:
  * date        (date) datetime64[ns] 2008-01-01 2008-01-02 ... 2020-12-31
  * gauge_id    (gauge_id) object '2177' '2166'
Data variables: (12/23)
    lvl_sm      (gauge_id, date) float64 167.0 168.0 167.0 ... 212.0 213.0 214.0
    q_cms_s     (gauge_id, date) float64 nan nan nan nan nan ... nan nan nan nan
    lvl_mbs     (gauge_id, date) float64 9.63 9.64 9.63 9.63 ... 5.12 5.13 5.14
    q_mm_day    (gauge_id, date) float64 nan nan nan nan nan ... nan nan nan nan
    t_max_e5l   (gauge_id, date) float64 -6.623 -7.65 -10.5 ... -9.92 -10.04
    t_max_e5    (gauge_id, date) float64 -5.787 -6.653 -10.81 ... -10.57 -9.54
    ...          ...
    SMroot      (gauge_id, date) float64 0.4046 0.4041 0.4039 ... 0.3507 0.3507
    Ew          (gauge_id, date) float64 1.666e-05 2.525e-05 ... 0.0002214
    Ei          (gauge_id, date) float64 0.1523 0.04446 0.1279 ... 0.0 0.0 0.0
    S           (gauge_id, date) float64 0.9743 0.9743 0.9743 ... 0.9729 0.9729
    E           (gauge_id, date) float64 0.1082 -0.004399 ... -0.04467 -0.01906
    Ep          (gauge_id, date) float64 0.1082 -0.004399 ... -0.04467 -0.01906

##### Select by gauge

In [11]:
gauges = ['2166']
ds = big_file.sel(gauge_id=gauges)
# sel is possible on date coordinate as well
ds

<xarray.Dataset>
Dimensions:     (date: 4749, gauge_id: 1)
Coordinates:
  * date        (date) datetime64[ns] 2008-01-01 2008-01-02 ... 2020-12-31
  * gauge_id    (gauge_id) object '2166'
Data variables: (12/23)
    lvl_sm      (gauge_id, date) float64 194.0 199.0 207.0 ... 212.0 213.0 214.0
    q_cms_s     (gauge_id, date) float64 nan nan nan nan nan ... nan nan nan nan
    lvl_mbs     (gauge_id, date) float64 4.94 4.99 5.07 5.11 ... 5.12 5.13 5.14
    q_mm_day    (gauge_id, date) float64 nan nan nan nan nan ... nan nan nan nan
    t_max_e5l   (gauge_id, date) float64 -2.021 -1.339 -3.978 ... -9.92 -10.04
    t_max_e5    (gauge_id, date) float64 -0.9709 -0.1157 -2.672 ... -10.57 -9.54
    ...          ...
    SMroot      (gauge_id, date) float64 0.3507 0.3507 0.3506 ... 0.3507 0.3507
    Ew          (gauge_id, date) float64 1.718e-05 1.822e-05 ... 0.0002214
    Ei          (gauge_id, date) float64 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    S           (gauge_id, date) float64 0.9729 0.9729 0.9729 ... 0.9729 0.9729
    E           (gauge_id, date) float64 -0.06727 -0.07272 ... -0.04467 -0.01906
    Ep          (gauge_id, date) float64 -0.06727 -0.07272 ... -0.04467 -0.01906

#### Select by other variables and split data

In [12]:
predictors = ['t_max_e5l', 't_max_e5', 't_min_e5l', 't_min_e5', 'prcp_e5l',
             'prcp_e5', 'prcp_gpcp', 'prcp_imerg', 'prcp_mswep', 'Eb', 'Es',
             'Et', 'SMsurf', 'SMroot', 'Ew', 'Ei', 'S', 'E', 'Ep']
target = ['lvl_sm', 'q_cms_s', 'lvl_mbs', 'q_mm_day']

future_window = 7
past_window = 365

train_start = '01/01/2008'
train_end = '12/31/2015'

val_start = '01/01/2016'
val_end = '12/31/2018'

test_start = '01/01/2019'
test_end = '12/31/2020'

In [13]:
ds.sel(date=slice(train_start, train_end))

<xarray.Dataset>
Dimensions:     (date: 2922, gauge_id: 1)
Coordinates:
  * date        (date) datetime64[ns] 2008-01-01 2008-01-02 ... 2015-12-31
  * gauge_id    (gauge_id) object '2166'
Data variables: (12/23)
    lvl_sm      (gauge_id, date) float64 194.0 199.0 207.0 ... 223.0 228.0 227.0
    q_cms_s     (gauge_id, date) float64 nan nan nan nan nan ... nan nan nan nan
    lvl_mbs     (gauge_id, date) float64 4.94 4.99 5.07 5.11 ... 5.23 5.28 5.27
    q_mm_day    (gauge_id, date) float64 nan nan nan nan nan ... nan nan nan nan
    t_max_e5l   (gauge_id, date) float64 -2.021 -1.339 -3.978 ... -12.62 -14.3
    t_max_e5    (gauge_id, date) float64 -0.9709 -0.1157 ... -12.77 -14.58
    ...          ...
    SMroot      (gauge_id, date) float64 0.3507 0.3507 0.3506 ... 0.3531 0.3523
    Ew          (gauge_id, date) float64 1.718e-05 1.822e-05 ... 1.318e-05
    Ei          (gauge_id, date) float64 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    S           (gauge_id, date) float64 0.9729 0.9729 0.9729 ... 0.9729 0.9729
    E           (gauge_id, date) float64 -0.06727 -0.07272 ... -0.06862 -0.05773
    Ep          (gauge_id, date) float64 -0.06727 -0.07272 ... -0.06862 -0.05773

In [15]:
ds.sel(date=slice(train_start, train_end),
       gauge_id='2166')[target]

<xarray.Dataset>
Dimensions:   (date: 2922)
Coordinates:
  * date      (date) datetime64[ns] 2008-01-01 2008-01-02 ... 2015-12-31
    gauge_id  <U4 '2166'
Data variables:
    lvl_sm    (date) float64 194.0 199.0 207.0 211.0 ... 220.0 223.0 228.0 227.0
    q_cms_s   (date) float64 nan nan nan nan nan nan ... nan nan nan nan nan nan
    lvl_mbs   (date) float64 4.94 4.99 5.07 5.11 5.1 ... 5.14 5.2 5.23 5.28 5.27
    q_mm_day  (date) float64 nan nan nan nan nan nan ... nan nan nan nan nan nan

### Static

In [17]:
geo_data = pd.read_csv(f'../geo_data/static_attributes/kamchatka/geo_vector.csv',
                       index_col='gauge_id')
geo_data

,for_pc_sse,crp_pc_sse,inu_pc_ult,ire_pc_sse,lka_pc_use,prm_pc_sse,pst_pc_sse,cly_pc_sav,slt_pc_sav,snd_pc_sav,kar_pc_sse,urb_pc_sse,gwt_cm_sav,lkv_mc_usu,rev_mc_usu,sgr_dk_sav,slp_dg_sav,ws_area,ele_mt_sav,m_bs
gauge_id,,,,,,,,,,,,,,,,,,,,
2166,20.951920,0.0,24.646430,0.0,0.245486,0.000000,0.0,15.500305,35.510670,49.977262,0.0,0.0,44.815667,4.092446,0.0,21.063817,1.012187,261.500832,44.207428,3.00
2177,55.099769,0.0,8.620497,0.0,0.028202,18.797626,0.0,6.456502,52.912878,40.675476,0.0,0.0,276.905789,1.677298,0.0,166.503802,12.654381,3613.763147,556.506970,7.96
